In [1]:
import numpy as np
import matplotlib.pyplot as plt
from helper import *
import scipy.sparse as sp
from test import load_data_sparse
from SGD import * 
from ALS import *
from Visualization import cv_result
from sklearn.linear_model import LinearRegression
from crossval import *

In [2]:
pred_lst = []

In [3]:
data_name = "data/47b05e70-6076-44e8-96da-2530dc2187de_data_train.csv"

ratings, pandas = load_data_sparse(data_name, False)

print(pandas.shape) # (1176952, 3)

stop_criterion = 1e-4
prediction = []
pred_lst = []

(1176952, 3)


In [25]:
# SGD
nb_feature = 40
lambda_ = 0.01
pred_SGD, _, _ = matrix_factorization_SGD_CV(ratings, nb_feature, lambda_, lambda_, stop_criterion)


learn the matrix factorization using SGD...


In [26]:
print(type(pred_SGD))
pred_lst.append(pred_SGD)
print(len(pred_lst))

<class 'numpy.ndarray'>
1


In [27]:
# ALS
nb_feature = 40
lambda_ = 0.1
pred_ALS, _, _ = ALS_CV(ratings, nb_feature, lambda_, lambda_, stop_criterion)


learn the matrix factorization using ALS...


In [28]:
pred_lst.append(pred_ALS)
print(len(pred_lst))

2


In [29]:
pred_lst_2 = pred_lst.copy()

In [30]:
len(pred_lst_2)

2

In [32]:
simple_models = False

In [33]:
if simple_models:
    # simple_models
    from simple_models import *
    print("learn global mean")
    pred_globalmean =  global_mean(ratings, 0)
    print(pred_globalmean.shape)
    pred_lst.append(pred_globalmean)

    print("learn user mean")
    pred_usermean = user_mean(ratings, 0)
    print(pred_usermean.shape)
    pred_lst.append(pred_usermean)

    print("learn item mean")
    pred_itemmean = baseline_item_mean(ratings, 0)
    print(pred_itemmean.shape)
    pred_lst.append(pred_itemmean)

In [34]:
def prepareBlending(ratings, list_pred):
    # determine position of given ratings
    nonzero_row, nonzero_col = ratings.nonzero()
    nonzero_train = list(zip(nonzero_row, nonzero_col))
    
    prediction = []
    prediction_model = []
    
    for pred_id, pred in enumerate(list_pred):
        print(pred_id)
        
        print(pred.shape)
        
        pred_tmp = []
        for i in nonzero_train:
            pred_tmp.append(pred.item(i))

        print(len(pred_tmp)) # (1176952)
        
        prediction_model.append(pred_tmp)
    
        if pred_id == 0:
            prediction = pred_tmp
        else:
            prediction = np.vstack((prediction,pred_tmp))          
        
        print(len(prediction))
        print(type(prediction))
        
    return prediction_model, prediction

In [35]:
prediction_m, prediction = prepareBlending(ratings, pred_lst)
print(prediction.shape)
print(len(prediction_m))

0
(10000, 1000)
1176952
1176952
<class 'list'>
1
(10000, 1000)
1176952
2
<class 'numpy.ndarray'>
(2, 1176952)
2


## Blending

In [33]:
y = pandas['Rating']
y.head()

,Rating,User,Movie
0,4,43,0
1,3,60,0
2,4,66,0
3,3,71,0
4,5,85,0


In [95]:
X = np.transpose(prediction)
X[0:5]

,0,1,2,3
0,4.482649,4.862356,4.601055,4.573068
1,4.116871,3.342292,3.151673,3.725260
2,4.291094,4.698996,4.326409,4.700248
3,4.303059,4.230530,4.184244,4.279365
4,2.721534,2.558059,2.535235,2.393988


#### Linear Regression

In [96]:
#clf = LinearRegression()
#clf = clf.fit(X, y)
#print('Weights of the different models:', clf.coef_)
#clf.score(X, y)

Weights of the different models: [ 0.00639413 -0.09418211  1.07471705  0.02613074]


#### Ridge Regression

In [97]:
from sklearn.linear_model import Ridge
linreg = Ridge(alpha=0.1, fit_intercept=False)
linreg.fit(X, y)
linreg.coef_

array([ 0.03225115, -0.09648427,  1.07089289,  0.0309938 ])

In [74]:
linreg.score(X, y)

-0.19198981056164754

In [21]:
# compute MSE - Blending
sum((np.round(X.dot(linreg.coef_)) - y)**2)/y.shape[0]

1.5979589651914436

In [22]:
# compute MSE - SGD
sum(np.round(prediction_m[0] - y)**2)/y.shape[0]

1.6271810575112664

In [23]:
# compute MSE - ALS
sum(np.round(prediction_m[1] - y)**2)/y.shape[0]

1.6172902548277246

## Manual comparison of ratings

In [98]:
print(X[0:10])

          0         1         2         3
0  4.482649  4.862356  4.601055  4.573068
1  4.116871  3.342292  3.151673  3.725260
2  4.291094  4.698996  4.326409  4.700248
3  4.303059  4.230530  4.184244  4.279365
4  2.721534  2.558059  2.535235  2.393988
5  3.548843  3.451663  3.169321  3.522247
6  3.611524  3.890638  3.503754  3.802356
7  4.418959  4.092229  3.650567  4.406050
8  3.915700  4.037428  3.955349  4.046888
9  3.652462  3.802900  3.586445  3.749890


In [99]:
print(y[0:10])

(4.0, 1.0, 4.0, 5.0, 2.0, 2.0, 3.0, 3.0, 4.0, 2.0)


In [100]:
# final result
result = X.dot(linreg.coef_)
print(result[0:10])

0    4.744404
1    3.300860
2    4.463813
3    4.344110
4    2.630124
5    3.284594
6    3.611085
7    3.793608
8    4.097921
9    3.707798
dtype: float64


In [28]:
if simple_models:
    result = pred_SGD * linreg.coef_[0] + pred_ALS * linreg.coef_[1] + pred_globalmean * linreg.coef_[2] + pred_usermean * linreg.coef_[3] + pred_itemmean * linreg.coef_[4]
else:
    result = pred_SGD * linreg.coef_[0] + pred_ALS * linreg.coef_[1]
        
    

In [105]:
result.shape

(1176952,)

In [30]:
if simple_models:
    create_submission_from_prediction(result, "out5-0.1.csv")
else:
    create_submission_from_prediction(result, "out2-0.1.csv")

In [106]:
create_submission_from_prediction(result, "out-4modelsurprise.csv")